In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd
import os
import copy
import numpy as np
import optuna
optuna.logging.disable_default_handler()
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
class MyDataSet(Dataset):
    def __init__(self, csv_path, root_dir):
        self.image_dataframe = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.images = os.listdir(self.root_dir)
        self.transform = transforms.Compose([transforms.ToTensor()])
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # 画像読み込み
        image_name = self.images[idx]
        image = Image.open(os.path.join(self.root_dir, image_name) )
        image = image.convert('RGB') # PyTorch 0.4以降
        # label (0 or 1)
        label = self.image_dataframe.query('ImageName=="'+image_name+'"')['ImageLabel'].iloc[0]
        return self.transform(image), int(label)

imgDataset = MyDataSet('/home/naoki/Documents/newgame.csv', '/home/naoki/Pictures/anime_face/newgame!!/data/')

In [7]:
train_data, test_data = train_test_split(imgDataset, test_size=0.2)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True, num_workers=4)

In [5]:
train_loader.num_workers

4

In [6]:
for data, target in train_loader:
    print(target.size())

torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([15])


In [31]:
image_name = imgDataset.images[8]
imgDataset.image_dataframe.query('ImageName=="'+image_name+'"')['ImageLabel']

131    4
Name: ImageLabel, dtype: int64

In [10]:
imgDataset.image_dataframe

,ImageName,ImageLabel
0,2.png,0
1,3.png,1
2,4.png,0
3,8.png,2
4,27.png,0
5,30.png,2
6,32.png,0
7,35.png,0
8,37.png,1
9,45.png,0


In [9]:
EPOCH = 10

class Net(nn.Module):
    def __init__(self, trial):
        super(Net, self).__init__()
        self.activation = get_activation(trial)
        # self.activation = trial.suggest_categorical('activation', [F.relu, F.elu])
        self.conv1 = nn.Conv2d(3, 30, kernel_size=5)  #  64*64*3 -> 60*60*30
        self.conv2 = nn.Conv2d(30, 60, kernel_size=5)  #  30*30*30 -> 26*26*60
        self.conv2_drop = nn.Dropout2d(p=trial.suggest_uniform("dropout_prob", 0, 0.8))  #  0〜0.8の間でサンプリング
        self.fc1 = nn.Linear(13*13*60, 150)
        self.fc2 = nn.Linear(150, 6)

    def forward(self, x):
        x = self.activation(F.max_pool2d(self.conv1(x), 2))
        x = self.activation(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 13*13*60)
        x = self.activation(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(model, device, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():  #  計算グラフを作らない
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    return 1 - correct / len(test_loader.dataset)

def get_optimizer(trial, model):
    optimizer_names = ['Adam', 'MomentumSGD']
    optimizer_name = trial.suggest_categorical('optimizer', optimizer_names)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-10, 1e-3)
    if optimizer_name == optimizer_names[0]: 
        adam_lr = trial.suggest_loguniform('adam_lr', 1e-5, 1e-1)
        optimizer = optim.Adam(model.parameters(), lr=adam_lr, weight_decay=weight_decay)
    else:
        momentum_sgd_lr = trial.suggest_loguniform('momentum_sgd_lr', 1e-5, 1e-1)
        optimizer = optim.SGD(model.parameters(), lr=momentum_sgd_lr,
                              momentum=0.9, weight_decay=weight_decay)
    return optimizer

# def adam(model, trial, weight_decay):
#     adam_lr = trial.suggest_loguniform('adam_lr', 1e-5, 1e-1)
#     return optim.Adam(model.parameters(), lr=adam_lr, weight_decay=weight_decay)
    
# def momentum(model, trial, weight_decay):
#     momentum_sgd_lr = trial.suggest_loguniform('momentum_sgd_lr', 1e-5, 1e-1)
#     return optim.SGD(model.parameters(), lr=momentum_sgd_lr,
#                      momentum=0.9, weight_decay=weight_decay)

# def get_optimizer(trial, model):
#     optimizer = trial.suggest_categorical('optimizer', [adam, momentum])
#     weight_decay = trial.suggest_loguniform('weight_decay', 1e-10, 1e-3)
#     return optimizer(model, trial, weight_decay)

def get_activation(trial):
    activation_names = ['ReLU', 'ELU']
    activation_name = trial.suggest_categorical('activation', activation_names)
    if activation_name == activation_names[0]:
        activation = F.relu
    else:
        activation = F.elu
    return activation

def objective_wrapper(pbar):
    def objective(trial):  #  optunaを使う場合にはtrialを引数にする必要あり
        device = "cuda" if torch.cuda.is_available() else "cpu"

        model = Net(trial).to(device)
        optimizer = get_optimizer(trial, model)

        for step in range(EPOCH):
            train(model, device, train_loader, optimizer)
            error_rate = test(model, device, test_loader)

            trial.report(error_rate, step)
            if trial.should_prune(step):
                pbar.update()
                raise optuna.structs.TrialPruned()

        pbar.update()

        return error_rate
    
    return objective

In [10]:
TRIAL_SIZE = 100
with tqdm(total=TRIAL_SIZE) as pbar:
    study = optuna.create_study(pruner=optuna.pruners.MedianPruner())  #  インスタンス生成
    study.optimize(objective_wrapper(pbar), n_trials=TRIAL_SIZE)  #  最適化　目的関数の返り値が小さくなるようにパラメータ探索　-> objective : 誤り率

#  studyでtrialの処理が行われる

Setting trial status as TrialState.FAIL because of the following error: RuntimeError('size mismatch, m1: [64 x 10140], m2: [1014 x 150] at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/THC/generic/THCTensorMathBlas.cu:249',)
Traceback (most recent call last):
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/optuna/study.py", line 400, in _run_trial
    result = func(trial)
  File "<ipython-input-9-5f9f04741407>", line 89, in objective
    train(model, device, train_loader, optimizer)
  File "<ipython-input-9-5f9f04741407>", line 28, in train
    output = model(data)
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/nn/modules/module.py", line 491, in __call__
    result = self.forward(*input, **kwargs)
  File "<ipython-input-9-5f9f04741407>", line 18, in forward
    x = self.activation(self.fc1(x))
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/nn/modules/module.py", line 491, in __call__

RuntimeError: size mismatch, m1: [64 x 10140], m2: [1014 x 150] at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/THC/generic/THCTensorMathBlas.cu:249
Setting trial status as TrialState.FAIL because of the following error: RuntimeError('size mismatch, m1: [64 x 10140], m2: [1014 x 150] at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/THC/generic/THCTensorMathBlas.cu:249',)
Traceback (most recent call last):
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/optuna/study.py", line 400, in _run_trial
    result = func(trial)
  File "<ipython-input-9-5f9f04741407>", line 89, in objective
    train(model, device, train_loader, optimizer)
  File "<ipython-input-9-5f9f04741407>", line 28, in train
    output = model(data)
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/nn/modules/module.py", line 491, in __call__
    result = self.forward(*input, **kwargs)
  File "<ipython-input-9-5f9f04741407>", line 18, in forward


RuntimeError: size mismatch, m1: [64 x 10140], m2: [1014 x 150] at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/THC/generic/THCTensorMathBlas.cu:249
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f3e1cdcec18>>
Traceback (most recent call last):
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/naoki/anaconda3/en

RuntimeError: size mismatch, m1: [64 x 10140], m2: [1014 x 150] at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/THC/generic/THCTensorMathBlas.cu:249
Setting trial status as TrialState.FAIL because of the following error: RuntimeError('size mismatch, m1: [64 x 10140], m2: [1014 x 150] at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/THC/generic/THCTensorMathBlas.cu:249',)
Traceback (most recent call last):
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/optuna/study.py", line 400, in _run_trial
    result = func(trial)
  File "<ipython-input-9-5f9f04741407>", line 89, in objective
    train(model, device, train_loader, optimizer)
  File "<ipython-input-9-5f9f04741407>", line 28, in train
    output = model(data)
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/nn/modules/module.py", line 491, in __call__
    result = self.forward(*input, **kwargs)
  File "<ipython-input-9-5f9f04741407>", line 18, in forward


Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/naoki/anaconda3/envs/tensorflow/lib/python

KeyboardInterrupt: 

In [13]:
1 - study.best_value

0.96

In [15]:
study.

FrozenTrial(trial_id=15, state=<TrialState.COMPLETE: 1>, value=0.040000000000000036, datetime_start=datetime.datetime(2019, 1, 5, 14, 0, 40, 410531), datetime_complete=datetime.datetime(2019, 1, 5, 14, 0, 43, 66237), params={'activation': 'ELU', 'dropout_prob': 0.345478067932839, 'optimizer': 'Adam', 'weight_decay': 1.4248031080514087e-06, 'adam_lr': 0.0002900399340813237}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.6799999999999999, 1: 0.4, 2: 0.17000000000000004, 3: 0.040000000000000036, 4: 0.06999999999999995, 5: 0.050000000000000044, 6: 0.10999999999999999, 7: 0.050000000000000044, 8: 0.050000000000000044, 9: 0.040000000000000036}, params_in_internal_repr={'activation': 1, 'dropout_prob': 0.345478067932839, 'optimizer': 0, 'weight_decay': 1.4248031080514087e-06, 'adam_lr': 0.0002900399340813237})

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd
import os
import copy
import numpy as np
import optuna
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

BATCHSIZE = 64

optuna.logging.disable_default_handler()


class MyDataSet(Dataset):
    def __init__(self, csv_path, root_dir):
        self.image_dataframe = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.images = os.listdir(self.root_dir)
        self.transform = transforms.Compose([transforms.ToTensor()])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # 画像読み込み
        image_name = self.images[idx]
        image = Image.open(os.path.join(self.root_dir, image_name))
        image = image.convert('RGB')  # PyTorch 0.4以降
        # label (0 or 1)
        label = self.image_dataframe.query('ImageName=="' + image_name + '"')['ImageLabel'].iloc[0]
        return self.transform(image), int(label)


imgDataset = MyDataSet('/home/naoki/Documents/newgame.csv', '/home/naoki/Pictures/anime_face/newgame!!/data/')

train_data, test_data = train_test_split(imgDataset, test_size=0.2)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True, num_workers=4)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.activation = F.elu
        # self.activation = trial.suggest_categorical('activation', [F.relu, F.elu])
        self.conv1 = nn.Conv2d(3, 30, kernel_size=5)  # 64*64*3 -> 60*60*30
        self.conv2 = nn.Conv2d(30, 60, kernel_size=5)  # 30*30*30 -> 26*26*60
        self.conv2_drop = nn.Dropout2d(p=0.345478067932839)  # 0〜0.8の間でサンプリング
        self.fc1 = nn.Linear(13 * 13 * 60, 150)
        self.fc2 = nn.Linear(150, 6)

    def forward(self, x):
        x = self.activation(F.max_pool2d(self.conv1(x), 2))
        x = self.activation(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 13 * 13 * 60)
        x = self.activation(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


def train(epoch, model, device, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
                   100. * batch_idx / len(train_loader), loss.data[0]))


def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():  # 計算グラフを作らない
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        1 - correct / len(test_loader.dataset), correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


device = "cuda" if torch.cuda.is_available() else "cpu"
model = Net().to(device)
print(device)
optimizer = optim.Adam(model.parameters(), lr=0.0002900399340813237, weight_decay=1.4248031080514087e-06)

for epoch in range(1, 100 + 1):
    train(epoch, model, device, train_loader, optimizer)
    test(model, device, test_loader)


cuda


/home/naoki/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:87: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/399 (0%)]	Loss: 1.804325
Train Epoch: 1 [64/399 (14%)]	Loss: 1.818276
Train Epoch: 1 [128/399 (29%)]	Loss: 1.643928
Train Epoch: 1 [192/399 (43%)]	Loss: 1.587973
Train Epoch: 1 [256/399 (57%)]	Loss: 1.517244
Train Epoch: 1 [320/399 (71%)]	Loss: 1.656107
Train Epoch: 1 [90/399 (86%)]	Loss: 1.770938

Test set: Average loss: 0.6600, Accuracy: 34/100 (34%)

Train Epoch: 2 [0/399 (0%)]	Loss: 1.509064
Train Epoch: 2 [64/399 (14%)]	Loss: 1.487850
Train Epoch: 2 [128/399 (29%)]	Loss: 1.416274
Train Epoch: 2 [192/399 (43%)]	Loss: 1.343437
Train Epoch: 2 [256/399 (57%)]	Loss: 1.343033
Train Epoch: 2 [320/399 (71%)]	Loss: 1.230462
Train Epoch: 2 [90/399 (86%)]	Loss: 1.296121

Test set: Average loss: 0.5300, Accuracy: 47/100 (47%)

Train Epoch: 3 [0/399 (0%)]	Loss: 1.166622
Train Epoch: 3 [64/399 (14%)]	Loss: 1.205175
Train Epoch: 3 [128/399 (29%)]	Loss: 1.040114
Train Epoch: 3 [192/399 (43%)]	Loss: 1.081881
Train Epoch: 3 [256/399 (57%)]	Loss: 1.066868
Train Epoch: 3 [320/399 (7

Train Epoch: 44 [0/399 (0%)]	Loss: 0.014514
Train Epoch: 44 [64/399 (14%)]	Loss: 0.005701
Train Epoch: 44 [128/399 (29%)]	Loss: 0.011874
Train Epoch: 44 [192/399 (43%)]	Loss: 0.012709
Train Epoch: 44 [256/399 (57%)]	Loss: 0.024167
Train Epoch: 44 [320/399 (71%)]	Loss: 0.011285
Train Epoch: 44 [90/399 (86%)]	Loss: 0.032752

Test set: Average loss: 0.0500, Accuracy: 95/100 (95%)

Train Epoch: 45 [0/399 (0%)]	Loss: 0.012828
Train Epoch: 45 [64/399 (14%)]	Loss: 0.007638
Train Epoch: 45 [128/399 (29%)]	Loss: 0.008891
Train Epoch: 45 [192/399 (43%)]	Loss: 0.006151
Train Epoch: 45 [256/399 (57%)]	Loss: 0.015825
Train Epoch: 45 [320/399 (71%)]	Loss: 0.012169
Train Epoch: 45 [90/399 (86%)]	Loss: 0.010101

Test set: Average loss: 0.0500, Accuracy: 95/100 (95%)

Train Epoch: 46 [0/399 (0%)]	Loss: 0.014506
Train Epoch: 46 [64/399 (14%)]	Loss: 0.009413
Train Epoch: 46 [128/399 (29%)]	Loss: 0.003790
Train Epoch: 46 [192/399 (43%)]	Loss: 0.013778
Train Epoch: 46 [256/399 (57%)]	Loss: 0.014734
Train E


Train Epoch: 87 [0/399 (0%)]	Loss: 0.002033
Train Epoch: 87 [64/399 (14%)]	Loss: 0.001287
Train Epoch: 87 [128/399 (29%)]	Loss: 0.002986
Train Epoch: 87 [192/399 (43%)]	Loss: 0.000966
Train Epoch: 87 [256/399 (57%)]	Loss: 0.001482
Train Epoch: 87 [320/399 (71%)]	Loss: 0.000871
Train Epoch: 87 [90/399 (86%)]	Loss: 0.001254

Test set: Average loss: 0.0400, Accuracy: 96/100 (96%)

Train Epoch: 88 [0/399 (0%)]	Loss: 0.001904
Train Epoch: 88 [64/399 (14%)]	Loss: 0.000884
Train Epoch: 88 [128/399 (29%)]	Loss: 0.001096
Train Epoch: 88 [192/399 (43%)]	Loss: 0.001741
Train Epoch: 88 [256/399 (57%)]	Loss: 0.001676
Train Epoch: 88 [320/399 (71%)]	Loss: 0.000704
Train Epoch: 88 [90/399 (86%)]	Loss: 0.001981

Test set: Average loss: 0.0400, Accuracy: 96/100 (96%)

Train Epoch: 89 [0/399 (0%)]	Loss: 0.001161
Train Epoch: 89 [64/399 (14%)]	Loss: 0.000253
Train Epoch: 89 [128/399 (29%)]	Loss: 0.001251
Train Epoch: 89 [192/399 (43%)]	Loss: 0.000889
Train Epoch: 89 [256/399 (57%)]	Loss: 0.002225
Train 

In [70]:
import time

def test_final(model, device, test_loader):
    model.eval()
    correct = 0
    name_list = ["aoba", "hajime", "hihumi", "kou", "rin", "yun"]
    transform = transforms.Compose([transforms.ToPILImage()])
    with torch.no_grad():  # 計算グラフを作らない
        for data, target in test_loader:
            data_org = copy.deepcopy(data)
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            for i in range(data.size()[0]):
                image = transform(data_org[i])
                image.show()
                time.sleep(3)
                print("target : {}".format(name_list[target[i]]))
                print("pred : {}".format(name_list[pred[i]]))

In [71]:
test_final(model, device, test_loader)

target : hihumi
pred : hihumi
target : kou
pred : kou
target : yun
pred : yun
target : hajime
pred : hajime
target : aoba
pred : aoba
target : hihumi
pred : hihumi
target : rin
pred : rin
target : yun
pred : yun
target : kou
pred : kou
target : aoba
pred : aoba
target : aoba
pred : aoba
target : aoba
pred : aoba
target : hihumi
pred : rin
target : aoba
pred : aoba
target : hihumi
pred : hajime
target : aoba
pred : aoba
target : yun
pred : yun
target : aoba
pred : aoba
target : kou
pred : kou
target : aoba
pred : aoba
target : hihumi
pred : hihumi
target : rin
pred : rin
target : rin
pred : rin
target : aoba
pred : aoba
target : aoba
pred : aoba
target : yun
pred : yun
target : kou
pred : kou
target : hajime
pred : hajime
target : yun
pred : yun
target : aoba
pred : aoba
target : yun
pred : yun
target : hajime
pred : hajime
target : aoba
pred : aoba
target : aoba
pred : aoba
target : rin
pred : rin
target : kou
pred : kou
target : yun
pred : yun
target : aoba
pred : aoba
target : yun
pr

In [44]:
image_name = "example"
image = Image.open("/home/naoki/Pictures/anime_movie/newgame!!/41.png")
image = image.convert('RGB')
image.show()

In [59]:

image.size()[0]

3